In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

# Загрузим датасет о качестве вина (Wine Quality)
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
wine_data = pd.read_csv(url, sep=';')

# Посмотрим на данные
print(wine_data.head())
print(wine_data.info())
print(wine_data.describe())

# Проверим наличие пропусков
print("\nКоличество пропусков в каждом столбце:")
print(wine_data.isnull().sum())

# Если есть пропуски, заполним их медианными значениями
if wine_data.isnull().sum().sum() > 0:
    imputer = SimpleImputer(strategy='median')
    wine_data = pd.DataFrame(imputer.fit_transform(wine_data), columns=wine_data.columns)

# Посмотрим на распределение целевой переменной (quality)
print("\nРаспределение классов:")
print(wine_data['quality'].value_counts())

# Преобразуем задачу в бинарную классификацию: хорошее вино (>=7) и обычное вино (<7)
wine_data['quality_binary'] = (wine_data['quality'] >= 7).astype(int)
print("\nРаспределение бинарных классов:")
print(wine_data['quality_binary'].value_counts())

# Разделим признаки и целевую переменную
X = wine_data.drop(['quality', 'quality_binary'], axis=1)
y = wine_data['quality_binary']

# Масштабирование признаков
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Разделение на обучающую и тестовую выборки (80% обучение, 20% тест)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

# Обучение модели KNN с произвольным K=5
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

# Оценка качества модели
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nТочность модели с K={k}: {accuracy:.4f}")
print("\nОтчет по классификации:")
print(classification_report(y_test, y_pred))
print("\nМатрица ошибок:")
print(confusion_matrix(y_test, y_pred))

# Поиск оптимального K с помощью GridSearchCV
param_grid = {'n_neighbors': list(range(1, 31))}

# Стратегия 1: K-fold кросс-валидация
kf = KFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=kf, scoring='accuracy')
grid_search.fit(X_train, y_train)

print("\nРезультаты GridSearchCV с KFold:")
print(f"Лучшее значение K: {grid_search.best_params_['n_neighbors']}")
print(f"Лучшая точность при кросс-валидации: {grid_search.best_score_:.4f}")

# Оценка качества лучшей модели на тестовых данных
best_knn_kfold = grid_search.best_estimator_
y_pred_best_kfold = best_knn_kfold.predict(X_test)
accuracy_best_kfold = accuracy_score(y_test, y_pred_best_kfold)
print(f"Точность лучшей модели на тестовой выборке (KFold): {accuracy_best_kfold:.4f}")

# Стратегия 2: Stratified K-fold кросс-валидация
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search_strat = GridSearchCV(KNeighborsClassifier(), param_grid, cv=skf, scoring='accuracy')
grid_search_strat.fit(X_train, y_train)

print("\nРезультаты GridSearchCV с StratifiedKFold:")
print(f"Лучшее значение K: {grid_search_strat.best_params_['n_neighbors']}")
print(f"Лучшая точность при кросс-валидации: {grid_search_strat.best_score_:.4f}")

# Оценка качества лучшей модели на тестовых данных
best_knn_strat = grid_search_strat.best_estimator_
y_pred_best_strat = best_knn_strat.predict(X_test)
accuracy_best_strat = accuracy_score(y_test, y_pred_best_strat)
print(f"Точность лучшей модели на тестовой выборке (StratifiedKFold): {accuracy_best_strat:.4f}")

# RandomizedSearchCV с StratifiedKFold
param_dist = {'n_neighbors': list(range(1, 31))}
random_search = RandomizedSearchCV(
    KNeighborsClassifier(),
    param_distributions=param_dist,
    n_iter=10,
    cv=skf,
    scoring='accuracy',
    random_state=42
)
random_search.fit(X_train, y_train)

print("\nРезультаты RandomizedSearchCV с StratifiedKFold:")
print(f"Лучшее значение K: {random_search.best_params_['n_neighbors']}")
print(f"Лучшая точность при кросс-валидации: {random_search.best_score_:.4f}")

# Оценка качества лучшей модели на тестовых данных
best_knn_random = random_search.best_estimator_
y_pred_best_random = best_knn_random.predict(X_test)
accuracy_best_random = accuracy_score(y_test, y_pred_best_random)
print(f"Точность лучшей модели на тестовой выборке (RandomizedSearchCV): {accuracy_best_random:.4f}")

# Сравнение метрик качества
print("\nСравнение метрик качества:")
print(f"Исходная модель (K={k}): {accuracy:.4f}")
print(f"Лучшая модель по GridSearchCV с KFold (K={grid_search.best_params_['n_neighbors']}): {accuracy_best_kfold:.4f}")
print(f"Лучшая модель по GridSearchCV с StratifiedKFold (K={grid_search_strat.best_params_['n_neighbors']}): {accuracy_best_strat:.4f}")
print(f"Лучшая модель по RandomizedSearchCV (K={random_search.best_params_['n_neighbors']}): {accuracy_best_random:.4f}")

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 